In [1]:
import tifffile
from pathlib import Path
import napari


In [2]:
image = "data/IHC_1/AD1867/CA1/Image1.lsm"

# Extract filename
file_path = Path(image)
filename = file_path.stem

# Read the image file
img = tifffile.imread(image)


In [3]:
img.shape

(3, 1024, 1024)